In [ ]:
import cv2
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation,Dense,Flatten,MaxPooling2D,BatchNormalization,Conv2D,MaxPool2D,Dropout,Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications import ResNet50


In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   validation_split=0.2,
                                   rotation_range=30,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   width_shift_range=0.3,
                                   height_shift_range=0.3,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory('../input/emotion-detection-fer/train',
                                                 target_size=(48,48),
                                                 color_mode='rgb',
                                                 batch_size = 128,
                                                 class_mode = 'categorical',
                                                 shuffle=True,subset = 'training')

validation_set = train_datagen.flow_from_directory('../input/emotion-detection-fer/test',
                                                 target_size=(48,48),
                                                 color_mode='rgb',
                                                 batch_size = 128,
                                                 class_mode = 'categorical',
                                                 shuffle=True,subset = 'validation')
test_set =test_datagen.flow_from_directory('../input/emotion-detection-fer/test',
                                                 target_size=(48,48),
                                                 color_mode='rgb',
                                                 batch_size = 128,
                                                 class_mode = 'categorical',
                                                 shuffle=True)

In [ ]:
def create_vgg(input_dims=(48,48,3)):
    vgg19 = VGG19(
        include_top=False, 
        
        input_shape=input_dims
    )
    img = Input(input_dims,dtype=tf.float32)
    x = vgg19(img)
    x = Flatten()(x)
    x = Dense(7,activation='softmax')(x)
    model = Model(img,x)
    
    return model
model = create_vgg()
model.summary()


In [ ]:
def focal_loss(alpha,gamma):
    def loss(y_true,y_pred):
        y_pred = K.clip(y_pred, 1e-5,1-1e-5)
        l = y_true*K.log(y_pred)
        l = alpha*((1-y_pred)**gamma)*l
        l = -K.sum(l,axis=-1)
        return l
    return loss
model.compile(optimizer = Adam(0.0001), metrics = ['acc'], loss = focal_loss(2.0,4.0))
history=model.fit(training_set,epochs=80,validation_data=validation_set)

In [ ]:
history_dict=history.history
import matplotlib.pyplot as plt

train_loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']

epochs = range(1, len(history_dict['acc']) + 1)

plt.plot(epochs, train_loss_values, 'bo', label='Training loss')
plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
train_acc = history_dict['acc']
val_acc = history_dict['val_acc']
plt.plot(epochs, train_acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
model.evaluate(test_set)

In [ ]:
model.save('trained.h5')